# Arbeiten mit Mistral-Modellen

## Einführung

In dieser Lektion geht es um:
- Einen Überblick über die verschiedenen Mistral-Modelle
- Die Anwendungsfälle und Einsatzszenarien der einzelnen Modelle verstehen
- Codebeispiele zeigen die besonderen Eigenschaften jedes Modells


## Die Mistral-Modelle

In dieser Lektion werden wir drei verschiedene Mistral-Modelle kennenlernen: **Mistral Large**, **Mistral Small** und **Mistral Nemo**.

Jedes dieser Modelle ist kostenlos auf dem Github Model-Marktplatz verfügbar. Der Code in diesem Notebook verwendet diese Modelle, um den Code auszuführen. Hier findest du weitere Informationen zur Nutzung von Github Models, um [mit KI-Modellen zu prototypisieren](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
Mistral Large 2 ist derzeit das Spitzenmodell von Mistral und wurde für den Unternehmenseinsatz entwickelt.

Das Modell ist ein Upgrade zum ursprünglichen Mistral Large und bietet
-  Größeres Kontextfenster – 128k statt 32k
-  Bessere Leistung bei Mathematik- und Programmieraufgaben – durchschnittlich 76,9 % Genauigkeit statt 60,4 %
-  Verbesserte mehrsprachige Leistung – unterstützte Sprachen sind: Englisch, Französisch, Deutsch, Spanisch, Italienisch, Portugiesisch, Niederländisch, Russisch, Chinesisch, Japanisch, Koreanisch, Arabisch und Hindi.

Mit diesen Funktionen überzeugt Mistral Large besonders bei
- *Retrieval Augmented Generation (RAG)* – dank des größeren Kontextfensters
- *Function Calling* – dieses Modell unterstützt native Funktionsaufrufe, was die Integration mit externen Tools und APIs ermöglicht. Diese Aufrufe können sowohl parallel als auch nacheinander in einer festgelegten Reihenfolge erfolgen.
- *Code-Generierung* – dieses Modell ist besonders stark bei der Generierung von Python-, Java-, TypeScript- und C++-Code.


### RAG-Beispiel mit Mistral Large 2


In diesem Beispiel verwenden wir Mistral Large 2, um ein RAG-Muster auf ein Textdokument anzuwenden. Die Frage ist auf Koreanisch verfasst und erkundigt sich nach den Aktivitäten des Autors vor dem Studium.

Es wird das Cohere Embeddings Model genutzt, um Embeddings sowohl für das Textdokument als auch für die Frage zu erstellen. In diesem Beispiel kommt das Python-Paket faiss als Vektorspeicher zum Einsatz.

Der Prompt, der an das Mistral-Modell gesendet wird, enthält sowohl die Frage als auch die abgerufenen Textabschnitte, die der Frage ähnlich sind. Das Modell liefert anschließend eine Antwort in natürlicher Sprache.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small ist ein weiteres Modell aus der Mistral-Familie, das zur Kategorie Premier/Enterprise gehört. Wie der Name schon sagt, handelt es sich hierbei um ein Small Language Model (SLM). Die Vorteile von Mistral Small sind:
- Kosteneinsparungen im Vergleich zu Mistral LLMs wie Mistral Large und NeMo – 80% Preisersparnis
- Geringe Latenz – schnellere Antworten als die LLMs von Mistral
- Flexibel – kann in verschiedenen Umgebungen eingesetzt werden und benötigt weniger Ressourcen

Mistral Small eignet sich besonders für:
- Textbasierte Aufgaben wie Zusammenfassungen, Sentiment-Analysen und Übersetzungen
- Anwendungen mit häufigen Anfragen, da das Modell besonders kosteneffizient ist
- Aufgaben mit niedriger Latenz im Bereich Code, wie z.B. Code-Reviews und Vorschläge


## Vergleich von Mistral Small und Mistral Large

Um die Unterschiede in der Latenz zwischen Mistral Small und Large zu zeigen, führe die folgenden Zellen aus.

Du solltest einen Unterschied in den Antwortzeiten von 3-5 Sekunden sehen. Achte außerdem auf die Länge und den Stil der Antworten bei demselben Prompt.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Im Vergleich zu den beiden anderen in dieser Lektion besprochenen Modellen ist Mistral NeMo das einzige kostenlose Modell mit einer Apache2-Lizenz.

Es gilt als Weiterentwicklung des früheren Open-Source-LLM von Mistral, Mistral 7B.

Weitere Merkmale des NeMo-Modells sind:

- *Effizientere Tokenisierung:* Dieses Modell verwendet den Tekken-Tokenizer anstelle des häufiger genutzten tiktoken. Dadurch wird eine bessere Leistung für mehr Sprachen und Code erreicht.

- *Finetuning:* Das Basismodell steht für Finetuning zur Verfügung. Das bietet mehr Flexibilität für Anwendungsfälle, in denen ein Feintuning erforderlich sein könnte.

- *Native Function Calling* – Wie Mistral Large wurde auch dieses Modell für das Ausführen von Funktionsaufrufen trainiert. Damit ist es eines der ersten Open-Source-Modelle, das diese Fähigkeit besitzt.


## Mistral NeMo

Im Vergleich zu den beiden anderen in dieser Lektion besprochenen Modellen ist Mistral NeMo das einzige kostenlose Modell mit einer Apache2-Lizenz.

Es gilt als Weiterentwicklung des früheren Open-Source-LLM von Mistral, Mistral 7B.

Weitere Merkmale des NeMo-Modells sind:

- *Effizientere Tokenisierung:* Dieses Modell verwendet den Tekken-Tokenizer anstelle des häufiger genutzten tiktoken. Dadurch wird eine bessere Leistung für mehr Sprachen und Code erreicht.

- *Finetuning:* Das Basismodell steht für Finetuning zur Verfügung. Das bietet mehr Flexibilität für Anwendungsfälle, in denen ein Finetuning erforderlich sein könnte.

- *Native Function Calling* – Wie Mistral Large wurde dieses Modell für das Ausführen von Funktionsaufrufen trainiert. Damit ist es eines der ersten Open-Source-Modelle, das diese Fähigkeit besitzt.


### Vergleich von Tokenizern

In diesem Beispiel schauen wir uns an, wie Mistral NeMo die Tokenisierung im Vergleich zu Mistral Large handhabt.

Beide Beispiele verwenden denselben Prompt, aber du solltest sehen, dass NeMo weniger Tokens zurückgibt als Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Lernen hört hier nicht auf, setze deine Reise fort

Nachdem du diese Lektion abgeschlossen hast, schau dir unsere [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) an, um dein Wissen über Generative KI weiter auszubauen!



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner Ausgangssprache gilt als maßgebliche Quelle. Für wichtige Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
